#### to do:

- how can we apply machine learning to our project
- prepare/do sentimental analysis
- ...

In [1]:
import pandas as pd
from googletrans import Translator
from transformers import MarianMTModel, MarianTokenizer
import sentencepiece

/Users/hannes_seidl/opt/anaconda3/envs/FDS_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/Users/hannes_seidl/Desktop/UA_Aveiro/WS24:25/FDS/FDS_Project/working_file_081224.csv')
df.head()

,url,timestamp,processed_url_text,newspaper,arquivo_url,title,filtered_text,text_length,Lemmas_filtered,Lemmas_str,sentiment,sentiment_bin,sentiment_score,sentiment_label,published_date,year,entities,filtered_entities,dominant_topic,topic_label
0,https://www.cmjornal.pt/mundo/detalhe/andre-ve...,20210125190801,andre ventura apelidado de trump portugues na ...,cmjornal.pt/,https://arquivo.pt/noFrame/replay/202101251908...,"André Ventura apelidado de ""Trump português"" n...",Jornal francês destaca ascensão da extrema dir...,1928,"['jornal', 'francês', 'destacar', 'ascensão', ...",jornal francês destacar ascensão extremo país ...,0.30,Slightly Positive,3.0,positive,2024-12-08 15:55:00,2021,"[('País', 'LOC'), ('Marcelo Rebelo de Sousa', ...","[('Le Parisien', 'PER'), ('André Ventura', 'PE...",1,Legal and Constitutional Affairs [1]
1,https://www.cmjornal.pt/politica/detalhe/debat...,20210120182636,debate tenso entre andre ventura e joao ferrei...,cmjornal.pt/,https://arquivo.pt/noFrame/replay/202101201826...,Debate tenso entre André Ventura e João Ferrei...,Os candidatos presidenciais João Ferreira e An...,3960,"['candidato', 'presidencial', 'joão', 'ferreir...",candidato presidencial joão ferreira protagoni...,0.00,Neutral,1.0,positive,2021-01-20 00:00:00,2021,"[('João Ferreira', 'PER'), ('André Ventura', '...","[('João Ferreira', 'PER'), ('André Ventura', '...",0,Political Leadership and Governance [0]
2,https://www.dn.pt/edicao-do-dia/01-jul-2021/ve...,20210701033951,ventura acusa familia coxi de inventar discrim...,dn.pt/,https://arquivo.pt/noFrame/replay/202107010339...,"Ventura acusa família Coxi de ""inventar discri...","Ventura acusa família Coxi de ""inventar discri...",394,"['acusar', 'família', 'coxi', 'inventar', 'dis...",acusar família coxi inventar discriminação aju...,0.00,Neutral,-3.0,negative,2021-07-01 00:00:00,2021,"[('Coxi', 'PER'), ('Chega', 'LOC'), ('Bairro d...","[('Coxi', 'PER')]",4,Election Campaigns and Social Issues [4]
3,https://www.dn.pt/edicao-do-dia/07-ago-2020/cr...,20200807084230,crianca negra cartaz em manifestacao do chega,dn.pt/,https://arquivo.pt/noFrame/replay/202008070842...,"Criança negra ""cartaz"" em manifestação do Chega","Criança negra ""cartaz"" em manifestação do Cheg...",346,"['criança', 'negro', 'cartaz', 'manifestação',...",criança negro cartaz manifestação menina negro...,0.00,Neutral,-1.0,negative,2020-08-07 00:00:00,2020,"[('Chega\n\n', 'MISC'), ('Menina', 'PER'), ('C...","[('Menina', 'PER')]",1,Legal and Constitutional Affairs [1]
4,https://www.dn.pt/edicao-do-dia/08-out-2019/os...,20200207064445,os ciganos de alvito votaram no chega ha quem ...,dn.pt/,https://arquivo.pt/noFrame/replay/202002070644...,Os ciganos de Alvito votaram no Chega? Há quem...,Os ciganos de Alvito votaram no Chega? Há quem...,279,"['cigano', 'alvito', 'votar', 'digar', 'extrem...",cigano alvito votar digar extrema-direita parl...,0.05,Slightly Positive,0.0,neutral,2020-02-07 00:00:00,2020,"[('Alvito', 'LOC'), ('Chega?', 'MISC'), ('Cheg...","[('André Ventura', 'PER')]",4,Election Campaigns and Social Issues [4]


In [3]:
# setting up marianMT-model for translation:

model_name = "Helsinki-NLP/opus-mt-pt-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate_text(text):
    """
    Translates a single text from Portuguese to English using MarianMT.

    Args:
        text (str): Text in Portuguese to translate.

    Returns:
        str: Translated text in English.
    """
    if pd.isnull(text):
        return None  # Skip translation for NaN values
    inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Apply the translation function to the 'filtered_text' column
df['en_filtered_text'] = df['filtered_text'].apply(translate_text)

# Display the updated DataFrame
df.head()

OSError: Helsinki-NLP/opus-mt-pt-en is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`